# INFORMACIÓN DEL PROYECTO

### TÍTULO

Ucrania: pérdidas rusas y el coste de la comida

### TEMA

Una comparativa de 5 datasets de tamaños variables sobre diversos aspectos de la guerra de Ucrania que tienen que ver con las pérdidas en el bando ruso, los refugiados y el precio de la comida en los puestos locales. El nexo de la mayoría son las fechas.

### HIPÓTESIS

Define aquí lo que será la hipótesis de tu proyecto.
Puede que tengas más de una, pero tu proyecto lo harás teniendo una principal hipótesis.

1) Se espera que el precio de la comida en Ucrania oscile en función del número de refugiados que huyen del país y de la cercanía al frente de los puestos de venta. Asimismo, un peor rendimiento por parte del bando ruso debería animar a los proveedores a vender en el país y aumentar la oferta.

Más refugiados, mayor lejanía del frente o más pérdidas rusas = precios más bajos

2) Con el paso del tiempo, aumentan las pérdidas materiales rusas y se reducen las pérdidas humanas. Ello puede ser indicativo de una creciente preocupación de la administración rusa por la opinión pública, con lo que, entre otras cosas, usa más drones o artillería y menos tropas terrestres para minimizar la cuenta de bajas.

Uso mayor de tropas mecanizadas = uso reducido de soldados rasos = más pérdidas materiales y menos muertos

3) El ejército ruso todavía se sirve de mucho material de la Unión Soviética, que, en comparación con el equipamiento moderno, tiende a ser capturado o destruido con mayor frecuencia.

Uso de material soviético = más destrucciones y más capturas

## OBTENCIÓN DE LOS DATOS

### DATASETS Y FUENTES ALTERNATIVAS DE DATOS

Incluye aquí una breve descripción del dataset o datasets de los que partirás para poder evaluar tu hipótesis. <br>
También incluye el origen de estos datos y su fuente.

Acumulativo de refugiados ucranianos por fecha y destino (fuente original: API sobre actualizaciones de la guerra de RapidAPI):
https://www.kaggle.com/datasets/anuragbantu/ukraine-invasion-refugee-data-2022

Precio de la comida en cada mercado de Ucrania por fecha y tipo. He añadido una columna y he apuntado, a mano, la longitud en términos de coordenadas (fuente: Google Maps), ya que es una buena aproximación para calcular la distancia respecto a la frontera este, que es donde se abre el frente (fuente original: Centre for Humanitarian Data de la Oficina de Naciones Unidas para la Coordinación de Asuntos Humanitarios):
https://data.humdata.org/dataset/wfp-food-prices-for-ukraine

Pérdidas rusas personales por fecha (fuente original: múltiples, ver en Kaggle; entre ellas, el Ejército y el Ministerio de Defensa del Ucrania):
https://www.kaggle.com/datasets/piterfm/2022-ukraine-russian-war

Pérdidas rusas materiales, según categorías generales, por fecha (fuente original: es el mismo Kaggle que el anterior):
https://www.kaggle.com/datasets/piterfm/2022-ukraine-russian-war

Pérdidas materiales rusas, incluyendo modelo y fabricante, sin fecha (fuente original: Oryxspioenkop, web alemana de análisis datos de defensa de código abierto e investigación sobre la guerra):
https://www.kaggle.com/datasets/piterfm/2022-ukraine-russia-war-equipment-losses-oryx


Muestra mediante un head() los principales datasets con los que vas a trabajar

In [ ]:
# Refugiados ucranianos (acumulativo) por fecha y destino:

import pandas as pd

df_refugees = pd.read_csv(r'src\data\Refugees.csv')

df_refugees = df_refugees[['date', 'country', 'individuals']]

df_refugees.head()


In [ ]:
# Precio de la comida en cada mercado de Ucrania por fecha y tipo + columna de distancias aproximadas de la frontera este:

import datetime

df_prices = pd.read_csv(r'src\data\Prices.csv', low_memory=False)

df_prices = df_prices.drop(index=0)

df_prices['date'] = pd.to_datetime(df_prices['date'], exact=False).dt.date

start_war = datetime.date(2022, 2, 24)

df_prices = df_prices[df_prices['date'] >= start_war]

df_prices.reset_index(drop=True, inplace=True)

df_prices = df_prices[['date', 'category', 'commodity', 'unit', 'usdprice', 'market']]

longitude = {'Kherson': '32.6169° E',
                'Dnipropetrovsk': '35.0462° E',
                'Vinnytsya': '28.4682° E',
                'Zhytomyr': '28.6587° E',
                'Poltava': '34.5514° E',
                'Mykolayiv': '31.9946° E',
                'Kyiv city': '30.5234° E',
                'Iv.-Frankivsk': '24.7111° E',
                'Volyn': '25.3254° E',
                'Zaporizhzhya': '35.1396° E',
                'Khmelnytskiy': '26.9871° E',
                'Chernivtsi': '25.9358° E',
                'Kirovohrad': '32.2623° E',
                'Lviv': '24.0297° E',
                'Zakarpattya': '22.2879° E',
                'Sumy': '34.7981° E',
                'Ternopyl': '25.5948° E',
                'Odesa': '30.7233° E',
                'Cherkasy': '32.0598° E',
                'Kyiv': '30.5234° E',
                'Rivne': '26.2516° E',
                'Chernihiv': '31.2893° E',
                'Kharkiv': '36.2304° E',
                'Donetsk': '37.8028° E'
            }

for key, value in longitude.items():
    df_prices.loc[df_prices['market'] == key, 'longitude'] = value

df_prices.head()


In [ ]:
# Pérdidas rusas personales por fecha:

df_personnel = pd.read_csv(r'src\data\Russia_losses_personnel(date).csv')

for index, casualties in enumerate(df_personnel['personnel']):
    if index != 0:
        previous_index = index - 1
        df_personnel.loc[[index], 'absolute personnel'] = int(df_personnel.loc[[index], 'personnel']) - int(df_personnel.loc[[previous_index], 'personnel'])
    else:
         df_personnel.loc[[index], 'absolute personnel'] = df_personnel.loc[[index], 'personnel']

df_personnel = df_personnel[['date', 'absolute personnel', 'personnel']]

df_personnel.head()


In [ ]:
# Pérdidas rusas materiales, según categorías generales, por fecha:

df_equipment = pd.read_csv(r'src\data\Russia_losses_equipment(date).csv')

df_equipment.drop(columns=['day', 'greatest losses direction'], inplace=True)

df_equipment.head()


In [ ]:
# Pérdidas materiales rusas (incluye modelo y fabricante), por fecha:

df_tech = pd.read_csv(r'src\data\Russia_losses_equipment(tech_details).csv')

for column in df_tech.columns[4:]:
    pd.to_numeric(df_tech[column])

df_tech['total captured'] = df_tech[['captured', 
                                    'captured and destroyed',
                                    'captured and stripped',
                                    'damaged and captured'
                                    ]].sum(axis=1)

df_tech['total not captured'] = df_tech[['abandoned', 
                                        'abandoned and destroyed',
                                        'damaged',
                                        'damaged and abandoned',
                                        'damaged beyond economical repair',
                                        'damaged by Bayraktar TB2',
                                        'destroyed',
                                        'destroyed by Bayraktar TB2',
                                        'destroyed by Bayraktar TB2 and Harpoon AShM',
                                        'sunk'
                                        ]].sum(axis=1)

df_tech = df_tech[['equipment', 'model', 'manufacturer', 'losses_total', 'total captured', 'total not captured']]

df_tech.head()
